In [1]:
import pandas as pd
import numpy as np
from pingouin import cronbach_alpha
from scipy.stats import pearsonr, ttest_ind, f_oneway   

In [7]:
df = pd.read_csv("./resultados_avaliacao.csv")

In [10]:
def extrai_resposta(df): 
    # Lista de heurísticas e mapeamento das respostas
    questoes = [
        'Visibilidade do estado do sistema', 
        'Correspondência entre o sistema e o mundo real', 
        'Controle e liberdade do usuário', 
        'Consistência e padrões', 
        'Prevenção de erros', 
        'Reconhecimento em vez de memorização', 
        'Flexibilidade e eficiência de uso', 
        'Design estético e minimalista', 
        'Ajude os usuários a reconhecer, diagnosticar e corrigir erros', 
        'Ajuda e documentação'
    ]
    
    mapeamento_respostas = {
        'Concordo Totalmente': 5,
        'Concordo': 4,
        'Indiferente': 3,
        'Discordo': 2,
        'Discordo Totalmente': 1
    }

    heuristicas_respostas = {heuristica: [] for heuristica in questoes}
    for heuristica in questoes:
        for coluna in df.columns:
            if heuristica in coluna:
                contagem_respostas = df[coluna].value_counts()
                respostas = []
                for categoria, valor in mapeamento_respostas.items():
                    if categoria in contagem_respostas:
                        respostas += [valor] * contagem_respostas[categoria]
                heuristicas_respostas[heuristica].append(respostas)

    for heuristica, respostas in heuristicas_respostas.items():
        print(f"\nAnálise para a heurística: {heuristica}")
        
        df_respostas_heuristica = pd.DataFrame(respostas).transpose()
        
        # alpha = cronbach_alpha(df_respostas_heuristica)
        # print(f"Alfa de Cronbach: {alpha} - Consistência interna {'boa' if alpha >= 0.7 else 'baixa'}")

        correlacoes = []
        for j in range(len(df_respostas_heuristica.columns)):
            for k in range(j + 1, len(df_respostas_heuristica.columns)):
                common_indices = df_respostas_heuristica[[j, k]].dropna().index
                if len(common_indices) >= 2:
                    corr, p_value = pearsonr(df_respostas_heuristica[j].loc[common_indices], df_respostas_heuristica[k].loc[common_indices])
                    correlacoes.append({
                        'Pergunta 1': j + 1,
                        'Pergunta 2': k + 1,
                        'Correlação': corr,
                        'p-valor': p_value
                    })
        print("Correlação de Pearson entre perguntas:")
        for cor in correlacoes:
            sig = "significativa" if cor['p-valor'] < 0.05 else "não significativa"
            print(f"Pergunta {cor['Pergunta 1']} e Pergunta {cor['Pergunta 2']}: Correlação = {cor['Correlação']}, p-valor = {cor['p-valor']} ({sig})")

        for j in range(len(df_respostas_heuristica.columns)):
            for k in range(j + 1, len(df_respostas_heuristica.columns)):
                grupo_1 = df_respostas_heuristica[j].dropna()
                grupo_2 = df_respostas_heuristica[k].dropna()
                if len(grupo_1) > 1 and len(grupo_2) > 1:
                    t_stat, p_val_ttest = ttest_ind(grupo_1, grupo_2)
                    sig = "significativa" if p_val_ttest < 0.05 else "não significativa"
                    print(f"t-Student entre perguntas {j+1} e {k+1}: t={t_stat}, p-valor={p_val_ttest} ({sig})")

        grupos_anova = [df_respostas_heuristica[col].dropna() for col in df_respostas_heuristica.columns]
        if len(grupos_anova) > 1:
            f_stat, p_val_anova = f_oneway(*grupos_anova)
            sig = "significativa" if p_val_anova < 0.05 else "não significativa"
            print(f"ANOVA entre perguntas da heurística '{heuristica}': F={f_stat}, p-valor={p_val_anova} ({sig})")

def cronbach_alpha(df):
    df = df.dropna(axis=1, how='all').dropna(axis=0, how='any')
    variancias = df.var(axis=0, ddof=1)
    variancia_total = df.sum(axis=1).var(ddof=1)
    n = df.shape[1]
    if variancia_total == 0:
        return 0
    alpha = (n / (n - 1)) * (1 - (variancias.sum() / variancia_total))
    return alpha


In [11]:
extrai_resposta(df)


Análise para a heurística: Visibilidade do estado do sistema
Correlação de Pearson entre perguntas:
Pergunta 1 e Pergunta 2: Correlação = 0.8974305715393169, p-valor = 1.395813109633675e-26 (significativa)
Pergunta 1 e Pergunta 3: Correlação = 0.9563227941643621, p-valor = 4.036560135332918e-39 (significativa)
Pergunta 2 e Pergunta 3: Correlação = 0.8982770488117734, p-valor = 1.059856054094496e-26 (significativa)
t-Student entre perguntas 1 e 2: t=0.24745954062296283, p-valor=0.8049100431973809 (não significativa)
t-Student entre perguntas 1 e 3: t=0.42239420502922564, p-valor=0.6733761874810384 (não significativa)
t-Student entre perguntas 2 e 3: t=0.23315198234986206, p-valor=0.815979139560211 (não significativa)
ANOVA entre perguntas da heurística 'Visibilidade do estado do sistema': F=0.10499507065395985, p-valor=0.9003755283682277 (não significativa)

Análise para a heurística: Correspondência entre o sistema e o mundo real
Correlação de Pearson entre perguntas:
Pergunta 1 e Per